In [1]:
import os
import pandas as pd


PROCESSED_DIR = "../data/processed/"

combined_path = os.path.join(PROCESSED_DIR, "combined_pr_pop.parquet")
combined_pr = pd.read_parquet(combined_path)

print("combined_pr shape:", combined_pr.shape)
combined_pr.head()

combined_pr shape: (40214, 14)


,agent,body,closed_at,created_at,html_url,id,merged_at,number,repo_url,state,title,user,user_id,author_type
0,Claude_Code,## Summary\n\nFixes an issue where `load_colle...,2025-07-29T07:01:20Z,2025-07-26 02:59:01+00:00,https://github.com/milvus-io/pymilvus/pull/2911,3264933329,None,2911,https://api.github.com/repos/milvus-io/pymilvus,closed,Fix: Wait for all partitions in load_collectio...,weiliu1031,108661493,agent
1,Claude_Code,## 背景\n\n現在、本プロジェクトにおいて以下のパス構成の不整合が生じています：\n\n...,2025-07-26T22:12:24Z,2025-07-26 04:56:55+00:00,https://github.com/classmethod/tsumiki/pull/2,3265118634,2025-07-26T22:12:24Z,2,https://api.github.com/repos/classmethod/tsumiki,closed,ファイルパス参照を相対パスに統一し、doc/からdocs/に統一,cm-kojimat,61827001,agent
2,Claude_Code,## Summary\r\n\r\n Implements comprehensive b...,2025-07-26T13:37:22Z,2025-07-26 13:31:19+00:00,https://github.com/steipete/Peekaboo/pull/30,3265640341,2025-07-26T13:37:22Z,30,https://api.github.com/repos/steipete/Peekaboo,closed,Add build staleness detection for debug CLI,MSch,7475,agent
3,Claude_Code,## Type of Change\n\n- [ ] 🐛 `bug` - Bug fix (...,2025-07-26T14:45:30Z,2025-07-26 14:07:22+00:00,https://github.com/sugyan/claude-code-webui/pu...,3265709660,2025-07-26T14:45:30Z,205,https://api.github.com/repos/sugyan/claude-cod...,closed,feat: add comprehensive README screenshots wit...,sugyan,80381,agent
4,Claude_Code,## Summary\r\n\r\n#17545 \r\n\r\nRemove the un...,None,2025-07-26 15:02:48+00:00,https://github.com/paradigmxyz/reth/pull/17625,3265782173,None,17625,https://api.github.com/repos/paradigmxyz/reth,open,chore: remove HashedPostStateProvider trait,adust09,47593288,agent


In [2]:
# Keep only PRs authored by autonomous coding agents
agent_prs = combined_pr[combined_pr["author_type"] == "agent"].copy()

print("Total agentic PRs in population:", len(agent_prs))
agent_prs[["id", "number", "state"]].head()

Total agentic PRs in population: 33596


,id,number,state
0,3264933329,2911,closed
1,3265118634,2,closed
2,3265640341,30,closed
3,3265709660,205,closed
4,3265782173,17625,open


In [3]:
# 1) Define a boolean 'merged' column if it's not already there
if "merged" not in agent_prs.columns:
    agent_prs["merged"] = agent_prs["merged_at"].notna()

# 2) Closed without merge = PR is closed and not merged
agent_prs["closed_without_merge"] = (
    (agent_prs["state"] == "closed") & (~agent_prs["merged"])
)

# 3) Quick look at first few rows
agent_prs[["id", "number", "state", "merged", "closed_without_merge"]].head(10) 

print("Merged PRs:", agent_prs["merged"].sum())
print("Closed without merge:", agent_prs["closed_without_merge"].sum())
print("Still open:", (agent_prs["state"] == "open").sum())

Merged PRs: 24014
Closed without merge: 7270
Still open: 2312


In [6]:
# STEP 4a: Load CI workflow runs (from Prof Taher)
workflows_path = os.path.join(PROCESSED_DIR, "agentic_prs_ci_output_worklfows.csv")
workflows = pd.read_csv(workflows_path)

print("workflows shape:", workflows.shape)
workflows.head()

workflows["conclusion"].value_counts()


workflows shape: (99976, 14)


conclusion
success            74883
failure            17108
skipped             6009
cancelled           1584
action_required      239
startup_failure      107
Name: count, dtype: int64

In [7]:
workflows.columns
workflows.head(3)

,author_type,pr_id,pr_number,pr_html_url,workflow_name,run_id,run_number,status,conclusion,started_at,finished_at,duration_seconds,workflow_run_url,commit_sha
0,agent,3264933329,2911,https://github.com/milvus-io/pymilvus/pull/2911,Code checker on pull request,16558516839,2481,completed,success,2025-07-28T02:22:24Z,2025-07-28T02:23:03Z,39.0,https://github.com/milvus-io/pymilvus/actions/...,f528ff4da627bba470791a66534fd47d7165ed8d
1,agent,3264933329,2911,https://github.com/milvus-io/pymilvus/pull/2911,Test on pull request,16558516847,3148,completed,failure,2025-07-28T02:22:24Z,2025-07-28T02:23:19Z,55.0,https://github.com/milvus-io/pymilvus/actions/...,f528ff4da627bba470791a66534fd47d7165ed8d
2,agent,3264933329,2911,https://github.com/milvus-io/pymilvus/pull/2911,Check proto on pull request,16558516849,2646,completed,success,2025-07-28T02:22:24Z,2025-07-28T02:23:11Z,47.0,https://github.com/milvus-io/pymilvus/actions/...,f528ff4da627bba470791a66534fd47d7165ed8d


In [8]:
# STEP 5: Summarize CI info per PR (one row per PR)
bad_conclusions = {"failure", "cancelled", "timed_out", "startup_failure", "action_required"}

ci_summary = (
    workflows
    .groupby("pr_id")["conclusion"]
    .agg(
        ci_runs="count",
        ci_any_failure=lambda s: s.isin(bad_conclusions).any(),
        ci_any_success=lambda s: (s == "success").any()
    )
    .reset_index()
)

print("ci_summary shape:", ci_summary.shape)
ci_summary.head()

ci_summary shape: (31068, 4)


,pr_id,ci_runs,ci_any_failure,ci_any_success
0,2256777953,4,False,True
1,2256824367,4,False,True
2,2256920866,8,False,True
3,2256956124,4,False,True
4,2257022379,11,False,True


In [11]:
# STEP 6: Merge CI summary into agent PRs
agent_prs_ci = agent_prs.merge(ci_summary, left_on="id", right_on="pr_id", how="left")

# Fill NA for PRs with no CI runs
agent_prs_ci["ci_runs"] = agent_prs_ci["ci_runs"].fillna(0).astype(int)
agent_prs_ci["ci_any_failure"] = agent_prs_ci["ci_any_failure"].fillna(False)
agent_prs_ci["ci_any_success"] = agent_prs_ci["ci_any_success"].fillna(False)

agent_prs_ci[
    ["id", "number", "state", "merged", "closed_without_merge",
     "ci_runs", "ci_any_failure", "ci_any_success"]
].head(10)


,id,number,state,merged,closed_without_merge,ci_runs,ci_any_failure,ci_any_success
0,3264933329,2911,closed,False,True,3,True,True
1,3265118634,2,closed,True,False,0,False,False
2,3265640341,30,closed,True,False,1,True,False
3,3265709660,205,closed,True,False,1,False,True
4,3265782173,17625,open,False,False,12,True,True
5,3231949586,32656,open,False,False,2,False,True
6,3231950376,32657,open,False,False,1,True,False
7,3231951012,32658,open,False,False,1,True,False
8,3231951656,32659,open,False,False,1,True,False
9,3231952364,32660,open,False,False,1,True,False


In [12]:

print("Agentic PRs with any CI runs:", (agent_prs_ci["ci_runs"] > 0).sum())
print("Agentic PRs with CI failure:", agent_prs_ci["ci_any_failure"].sum())
print("Agentic PRs with CI success:", agent_prs_ci["ci_any_success"].sum())

Agentic PRs with any CI runs: 25815
Agentic PRs with CI failure: 8660
Agentic PRs with CI success: 20721


In [13]:
# STEP 7: Mark failure candidates for RQ1
agent_prs_ci["failure_candidate"] = (
    agent_prs_ci["closed_without_merge"] | agent_prs_ci["ci_any_failure"]
)

failure_candidates = agent_prs_ci[agent_prs_ci["failure_candidate"]].copy()

print("Total agentic PRs:", len(agent_prs_ci))
print("Failure candidates:", len(failure_candidates))

failure_candidates[
    ["id", "number", "state", "merged",
     "closed_without_merge", "ci_runs",
     "ci_any_failure", "ci_any_success"]
].head(10)

Total agentic PRs: 33596
Failure candidates: 13601


,id,number,state,merged,closed_without_merge,ci_runs,ci_any_failure,ci_any_success
0,3264933329,2911,closed,False,True,3,True,True
2,3265640341,30,closed,True,False,1,True,False
4,3265782173,17625,open,False,False,12,True,True
6,3231950376,32657,open,False,False,1,True,False
7,3231951012,32658,open,False,False,1,True,False
8,3231951656,32659,open,False,False,1,True,False
9,3231952364,32660,open,False,False,1,True,False
10,3231953027,32661,open,False,False,2,True,True
11,3231953757,32662,open,False,False,2,True,True
12,3231954470,32663,open,False,False,2,True,True


In [14]:
from urllib.parse import urlparse

def extract_repo_full_name(url: str):
    if pd.isna(url):
        return None
    path = urlparse(url).path        # e.g. "/milvus-io/pymilvus/pull/2911"
    parts = path.strip("/").split("/")  # ["milvus-io", "pymilvus", "pull", "2911"]
    if len(parts) >= 2:
        return f"{parts[0]}/{parts[1]}"
    return None

agent_prs_ci["repo_full_name"] = agent_prs_ci["html_url"].apply(extract_repo_full_name)

agent_prs_ci[["id", "number", "repo_full_name"]].head()

,id,number,repo_full_name
0,3264933329,2911,milvus-io/pymilvus
1,3265118634,2,classmethod/tsumiki
2,3265640341,30,steipete/Peekaboo
3,3265709660,205,sugyan/claude-code-webui
4,3265782173,17625,paradigmxyz/reth


In [15]:
failure_candidates = agent_prs_ci[agent_prs_ci["failure_candidate"]].copy()

failure_by_repo = (
    failure_candidates["repo_full_name"]
    .value_counts()
    .reset_index()
    .rename(columns={"index": "repo_full_name", "repo_full_name": "n_failure_prs"})
)

print("Number of repos with at least one failing Agentic-PR:", len(failure_by_repo))
failure_by_repo.head(20)

Number of repos with at least one failing Agentic-PR: 1781


,n_failure_prs,count
0,mochilang/mochi,2178
1,MontrealAI/AGI-Alpha-Agent-v0,2178
2,crewAIInc/crewAI,287
3,jdereg/java-util,224
4,wieslawsoltes/Dock,184
5,calcom/cal.com,174
6,KomodoPlatform/komodo-wallet,140
7,MihaiCristianCondrea/Smart-Cleaner-for-Android,137
8,carverauto/serviceradar,137
9,airbytehq/airbyte,137


In [16]:
import numpy as np

target_total = 100      # size of your manual coding sample
max_per_repo = 10       # cap per repo

sampled_rows = []

for repo_name, group in failure_candidates.groupby("repo_full_name"):
    n_in_repo = len(group)
    n_take = min(max_per_repo, n_in_repo)
    sampled = group.sample(n=n_take, random_state=42)
    sampled_rows.append(sampled)

sampled_prs = pd.concat(sampled_rows, ignore_index=True)

print("Sampled before global cap:", len(sampled_prs))

if len(sampled_prs) > target_total:
    sampled_prs = sampled_prs.sample(n=target_total, random_state=42)

print("Final sampled PRs:", len(sampled_prs))

sampled_prs[
    ["id", "number", "repo_full_name", "state",
     "merged", "closed_without_merge",
     "ci_runs", "ci_any_failure", "ci_any_success"]
].head()

Sampled before global cap: 5243
Final sampled PRs: 100


,id,number,repo_full_name,state,merged,closed_without_merge,ci_runs,ci_any_failure,ci_any_success
1858,2758595312,299,different-ai/note-companion,closed,True,False,1,True,False
3538,3238853538,7323,microsoft/vscode-pull-request-github,closed,False,True,0,False,False
1128,2964834156,56960,airbytehq/airbyte,closed,False,True,2,True,False
3143,3227896964,15250,mautic/mautic,closed,False,True,0,False,False
931,3244652867,233,StephenDev0/StikDebug,closed,False,True,1,False,True


In [17]:
def make_pr_url(row):
    # html_url already points to the GitHub PR page
    return row["html_url"]

sampled_prs["pr_url"] = sampled_prs.apply(make_pr_url, axis=1)

# Add empty coding columns for your failure modes
for col in [
    "T1_ci_test_failure",
    "T2_logic_bug",
    "T3_syntax_compile",
    "T4_missing_tests",
    "T5_style_conventions",
    "T6_build_ci_config",
    "P1_low_priority",
    "P2_duplicate_alt_solution",
    "P3_abandoned_no_response",
    "P4_unknown",
    "primary_failure_mode",
    "notes",
]:
    sampled_prs[col] = ""

coding_cols = [
    "id", "number", "repo_full_name", "state", "merged", "closed_without_merge",
    "ci_runs", "ci_any_failure", "ci_any_success", "pr_url",
    "T1_ci_test_failure", "T2_logic_bug", "T3_syntax_compile",
    "T4_missing_tests", "T5_style_conventions", "T6_build_ci_config",
    "P1_low_priority", "P2_duplicate_alt_solution",
    "P3_abandoned_no_response", "P4_unknown",
    "primary_failure_mode", "notes",
]

coding_sheet = sampled_prs[coding_cols]
coding_sheet.head()

,id,number,repo_full_name,state,merged,closed_without_merge,ci_runs,ci_any_failure,ci_any_success,pr_url,...,T3_syntax_compile,T4_missing_tests,T5_style_conventions,T6_build_ci_config,P1_low_priority,P2_duplicate_alt_solution,P3_abandoned_no_response,P4_unknown,primary_failure_mode,notes
1858,2758595312,299,different-ai/note-companion,closed,True,False,1,True,False,https://github.com/different-ai/note-companion...,...,,,,,,,,,,
3538,3238853538,7323,microsoft/vscode-pull-request-github,closed,False,True,0,False,False,https://github.com/microsoft/vscode-pull-reque...,...,,,,,,,,,,
1128,2964834156,56960,airbytehq/airbyte,closed,False,True,2,True,False,https://github.com/airbytehq/airbyte/pull/56960,...,,,,,,,,,,
3143,3227896964,15250,mautic/mautic,closed,False,True,0,False,False,https://github.com/mautic/mautic/pull/15250,...,,,,,,,,,,
931,3244652867,233,StephenDev0/StikDebug,closed,False,True,1,False,True,https://github.com/StephenDev0/StikDebug/pull/233,...,,,,,,,,,,


In [18]:
out_path = os.path.join(PROCESSED_DIR, "agentic_failure_modes_coding_sheet.csv")
coding_sheet.to_csv(out_path, index=False)
print("Saved coding sheet to:", out_path)

Saved coding sheet to: ../data/processed/agentic_failure_modes_coding_sheet.csv
